In [1]:
import numpy as np
import pandas as pd
import datetime
import pickle
import itertools
import time

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import manhattan_distances,pairwise_distances
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC

from metric_learn import NCA

from sklearn.metrics import roc_auc_score

from sklearn.model_selection import RandomizedSearchCV
from imblearn.over_sampling import SMOTE

In [2]:
X_train = pd.read_csv('X_train_unique.csv')
X_val = pd.read_csv('X_val_unique.csv')
X_test = pd.read_csv('X_test_unique.csv')

y_train = pd.read_csv('y_train_unique.csv')
y_val = pd.read_csv('y_val_unique.csv')

In [3]:
X_train.set_index('unique_id',inplace=True)
X_val.set_index('unique_id',inplace=True)
X_test.set_index('unique_id',inplace=True)
y_train.set_index('unique_id',inplace=True)
y_val.set_index('unique_id',inplace=True)

In [4]:
X_train_experienced = X_train[X_train['experience_flag']==1]
y_train_experienced = y_train[y_train.index.isin(list(X_train_experienced.index))]
X_val_experienced = X_val[X_val['experience_flag']==1]
y_val_experienced = y_val[y_val.index.isin(list(X_val_experienced.index))]
X_test_experienced = X_test[X_test['experience_flag']==1]

X_train_cold = X_train[X_train['experience_flag']==0]
y_train_cold = y_train[y_train.index.isin(list(X_train_cold.index))]
X_val_cold = X_val[X_val['experience_flag']==0]
y_val_cold = y_val[y_val.index.isin(list(X_val_cold.index))]
X_test_cold = X_test[X_test['experience_flag']==0]

In [5]:
X_train.shape

(4494, 1539)

### Undersampling

In [5]:
y_train_0 = y_train[y_train['female_label']==0]
y_train_1 = y_train[y_train['female_label']==1]

y_train_1['rand'] = np.random.rand(y_train_1.shape[0])
y_train_1.sort_values(by='rand',inplace=True)

y_train_1 = y_train_1.head((y_train.count()-y_train.sum()).values[0])

y_train_under = pd.concat([y_train_1[['female_label']],y_train_0])
y_train_under.sort_index(inplace=True)

X_train_under = X_train[X_train.index.isin(list(y_train_under.index))]

X_train_under.sort_index(inplace=True)

<ipython-input-5-389cc6b01bca>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train_1['rand'] = np.random.rand(y_train_1.shape[0])
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:6393: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

In [6]:
#X_train_under.to_csv('X_train_under.csv')
#y_train_under.to_csv('y_train_under.csv')

In [6]:
X_train_under = pd.read_csv('X_train_under.csv')
y_train_under = pd.read_csv('y_train_under.csv')

In [7]:
X_train_under.set_index('unique_id',inplace=True)
y_train_under.set_index('unique_id',inplace=True)

In [8]:
y_train_under['female_label'].sum()/y_train_under['female_label'].count()

0.5

* Undersampling of experienced

In [8]:
y_train_experienced_0 = y_train[y_train['female_label']==0]
y_train_experienced_1 = y_train[y_train['female_label']==1]

y_train_experienced_1['rand'] = np.random.rand(y_train_experienced_1.shape[0])
y_train_experienced_1.sort_values(by='rand',inplace=True)

y_train_experienced_1 = y_train_experienced_1.head((y_train.count()-y_train.sum()).values[0])

y_train_experienced_under = pd.concat([y_train_experienced_1[['female_label']],y_train_experienced_0])
y_train_experienced_under.sort_index(inplace=True)

X_train_experienced_under = X_train[X_train.index.isin(list(y_train_experienced_under.index))]

X_train_experienced_under.sort_index(inplace=True)

<ipython-input-8-48ba58a89b67>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train_experienced_1['rand'] = np.random.rand(y_train_experienced_1.shape[0])
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:6393: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

In [9]:
X_train_experienced_under.shape

(3104, 1539)

In [11]:
#X_train_experienced_under.to_csv('X_train_experienced_under.csv')
#y_train_experienced_under.to_csv('y_train_experienced_under.csv')

In [6]:
X_train_experienced_under = pd.read_csv('X_train_experienced_under.csv')
y_train_experienced_under = pd.read_csv('y_train_experienced_under.csv')

X_train_experienced_under.set_index('unique_id',inplace=True)
y_train_experienced_under.set_index('unique_id',inplace=True)

y_train_experienced_under['female_label'].sum()/y_train_experienced_under['female_label'].count()

0.5

In [7]:
X_train_experienced_under.shape

(3104, 1539)

* Undersampling of cold

In [11]:
y_train_cold_0 = y_train[y_train['female_label']==0]
y_train_cold_1 = y_train[y_train['female_label']==1]

y_train_cold_1['rand'] = np.random.rand(y_train_cold_1.shape[0])
y_train_cold_1.sort_values(by='rand',inplace=True)

y_train_cold_1 = y_train_cold_1.head((y_train.count()-y_train.sum()).values[0])

y_train_cold_under = pd.concat([y_train_cold_1[['female_label']],y_train_cold_0])
y_train_cold_under.sort_index(inplace=True)

X_train_cold_under = X_train[X_train.index.isin(list(y_train_cold_under.index))]

X_train_cold_under.sort_index(inplace=True)

<ipython-input-11-901d03f555a3>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_train_cold_1['rand'] = np.random.rand(y_train_cold_1.shape[0])


In [14]:
#X_train_cold_under.to_csv('X_train_cold_under.csv')
#y_train_cold_under.to_csv('y_train_cold_under.csv')

In [12]:
X_train_cold_under = pd.read_csv('X_train_cold_under.csv')
y_train_cold_under = pd.read_csv('y_train_cold_under.csv')

X_train_cold_under.set_index('unique_id',inplace=True)
y_train_cold_under.set_index('unique_id',inplace=True)

y_train_cold_under['female_label'].sum()/y_train_cold_under['female_label'].count()

0.5

In [16]:
model = LGBMClassifier()
model.fit(X_train,y_train)

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LGBMClassifier()

In [17]:
def performance_df(model):
    
    pred_train = pd.DataFrame(model.predict_proba(X_train)[:,1],columns=['pred'],index=X_train.index)
    pred_val = pd.DataFrame(model.predict_proba(X_val)[:,1],columns=['pred'],index=X_val.index)
    pred_train = pd.merge(pred_train,y_train,how='left',left_index=True, right_index=True)
    pred_val = pd.merge(pred_val,y_val,how='left',left_index=True, right_index=True)

    pred_train['pred_binary'] = np.where(pred_train['pred']>0.5,1,0)
    pred_val['pred_binary'] = np.where(pred_val['pred']>0.5,1,0)

    roc_train = roc_auc_score(pred_train['female_label'],pred_train['pred'])
    roc_val = roc_auc_score(pred_val['female_label'],pred_val['pred'])

    err_rate1_train = 1-(pred_train[pred_train['female_label']==1]['pred_binary'].sum()/pred_train[pred_train['female_label']==1]['pred_binary'].count())
    err_rate0_train = pred_train[pred_train['female_label']==0]['pred_binary'].sum()/pred_train[pred_train['female_label']==0]['pred_binary'].count()
    err_rate_train = err_rate1_train+err_rate0_train

    err_rate1_val = 1-(pred_val[pred_val['female_label']==1]['pred_binary'].sum()/pred_val[pred_val['female_label']==1]['pred_binary'].count())
    err_rate0_val = pred_val[pred_val['female_label']==0]['pred_binary'].sum()/pred_val[pred_val['female_label']==0]['pred_binary'].count()
    err_rate_val = err_rate1_val+err_rate0_val

    pred_train_experienced = pd.DataFrame(model.predict_proba(X_train_experienced)[:,1],columns=['pred'],index=X_train_experienced.index)
    pred_val_experienced = pd.DataFrame(model.predict_proba(X_val_experienced)[:,1],columns=['pred'],index=X_val_experienced.index)
    pred_train_experienced = pd.merge(pred_train_experienced,y_train_experienced,how='left',left_index=True, right_index=True)
    pred_val_experienced = pd.merge(pred_val_experienced,y_val_experienced,how='left',left_index=True, right_index=True)

    pred_train_experienced['pred_binary'] = np.where(pred_train_experienced['pred']>0.5,1,0)
    pred_val_experienced['pred_binary'] = np.where(pred_val_experienced['pred']>0.5,1,0)

    roc_train_experienced = roc_auc_score(pred_train_experienced['female_label'],pred_train_experienced['pred'])
    roc_val_experienced = roc_auc_score(pred_val_experienced['female_label'],pred_val_experienced['pred'])

    err_rate1_train_experienced = 1-(pred_train_experienced[pred_train_experienced['female_label']==1]['pred_binary'].sum()/pred_train_experienced[pred_train_experienced['female_label']==1]['pred_binary'].count())
    err_rate0_train_experienced = pred_train_experienced[pred_train_experienced['female_label']==0]['pred_binary'].sum()/pred_train_experienced[pred_train_experienced['female_label']==0]['pred_binary'].count()
    err_rate_train_experienced = err_rate1_train_experienced+err_rate0_train_experienced

    err_rate1_val_experienced = 1-(pred_val_experienced[pred_val_experienced['female_label']==1]['pred_binary'].sum()/pred_val_experienced[pred_val_experienced['female_label']==1]['pred_binary'].count())
    err_rate0_val_experienced = pred_val_experienced[pred_val_experienced['female_label']==0]['pred_binary'].sum()/pred_val_experienced[pred_val_experienced['female_label']==0]['pred_binary'].count()
    err_rate_val_experienced = err_rate1_val_experienced+err_rate0_val_experienced

    pred_train_cold = pd.DataFrame(model.predict_proba(X_train_cold)[:,1],columns=['pred'],index=X_train_cold.index)
    pred_val_cold = pd.DataFrame(model.predict_proba(X_val_cold)[:,1],columns=['pred'],index=X_val_cold.index)
    pred_train_cold = pd.merge(pred_train_cold,y_train_cold,how='left',left_index=True, right_index=True)
    pred_val_cold = pd.merge(pred_val_cold,y_val_cold,how='left',left_index=True, right_index=True)

    pred_train_cold['pred_binary'] = np.where(pred_train_cold['pred']>0.5,1,0)
    pred_val_cold['pred_binary'] = np.where(pred_val_cold['pred']>0.5,1,0)

    roc_train_cold = roc_auc_score(pred_train_cold['female_label'],pred_train_cold['pred'])
    roc_val_cold = roc_auc_score(pred_val_cold['female_label'],pred_val_cold['pred'])

    err_rate1_train_cold = 1-(pred_train_cold[pred_train_cold['female_label']==1]['pred_binary'].sum()/pred_train_cold[pred_train_cold['female_label']==1]['pred_binary'].count())
    err_rate0_train_cold = pred_train_cold[pred_train_cold['female_label']==0]['pred_binary'].sum()/pred_train_cold[pred_train_cold['female_label']==0]['pred_binary'].count()
    err_rate_train_cold = err_rate1_train_cold+err_rate0_train_cold

    err_rate1_val_cold = 1-(pred_val_cold[pred_val_cold['female_label']==1]['pred_binary'].sum()/pred_val_cold[pred_val_cold['female_label']==1]['pred_binary'].count())
    err_rate0_val_cold = pred_val_cold[pred_val_cold['female_label']==0]['pred_binary'].sum()/pred_val_cold[pred_val_cold['female_label']==0]['pred_binary'].count()
    err_rate_val_cold = err_rate1_val_cold+err_rate0_val_cold

    perf_df = pd.DataFrame({'Train ROC':[roc_train],
                            'Val ROC':[roc_val],
                            'Train BAR':[1-err_rate_train],
                            'Val BAR':[1-err_rate_val],
                            'Train ROC - Exp':[roc_train_experienced],
                            'Val ROC - Exp':[roc_val_experienced],
                            'Train BAR - Exp':[1-err_rate_train_experienced],
                            'Val BAR - Exp':[1-err_rate_val_experienced],
                            'Train ROC - Cold':[roc_train_cold],
                            'Val ROC - Cold':[roc_val_cold],
                            'Train BAR - Cold':[1-err_rate_train_cold],
                            'Val BAR - Cold':[1-err_rate_val_cold]})
    
    return perf_df

### LightGBM

In [18]:
lgbm_models_v02 = pd.DataFrame()

for under in [0,1]:
    for model_sample in ['all','exp','cold']:
        for max_d in [-1,2,4,10]:
            for lr in [0.01,0.1,0.5]:
                for n_est in [100,200]:
                    for min_c in [20,50,100]:
                        for cols in [0.5,0.8,1]:
                            
                            classifier = LGBMClassifier(max_depth=max_d,learning_rate=lr,n_estimators=n_est,min_child_samples=min_c,colsample_bytree=cols)
                            if model_sample == 'all':
                                if under == 0:
                                    classifier.fit(X_train,y_train)
                                else:
                                    classifier.fit(X_train_under,y_train_under)
                            elif model_sample == 'exp':
                                if under == 0:
                                    classifier.fit(X_train_experienced,y_train_experienced)
                                else:
                                    classifier.fit(X_train_experienced_under,y_train_experienced_under)
                            elif model_sample == 'cold':
                                if under == 0:
                                    classifier.fit(X_train_cold,y_train_cold)
                                else:
                                    classifier.fit(X_train_cold_under,y_train_cold_under)
                                    
                            perf_df_tmp = performance_df(classifier)
                            perf_df_tmp['Undersampling Flg'] = under
                            perf_df_tmp['Model Sample'] = model_sample
                            
                            perf_df_tmp['Max Depth'] = max_d
                            perf_df_tmp['Learning Rate'] = lr
                            perf_df_tmp['N Estimators'] = n_est
                            perf_df_tmp['Min Samples Leaf'] = min_c
                            perf_df_tmp['Columns Subsample'] = cols
                                    
                            lgbm_models_v02 = pd.concat([lgbm_models_v02,perf_df_tmp])
                                    
lgbm_models_v02.to_csv('lgbm_models_v02.csv')

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-

...

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-

### XGBoost

In [19]:
xgb_models_v02 = pd.DataFrame()

for under in [0,1]:
    for model_sample in ['all','exp','cold']:
        for max_d in [2,4,10]:
            for lr in [0.01,0.1,0.5]:
                for n_est in [100,200]:
                    for min_c in [20,50,100]:
                        for cols in [0.5,0.8,1]:
                            
                            classifier = XGBClassifier(max_depth=max_d,eta=lr,n_estimators=n_est,min_child_samples=min_c,colsample_bytree=cols)
                            if model_sample == 'all':
                                if under == 0:
                                    classifier.fit(X_train,y_train)
                                else:
                                    classifier.fit(X_train_under,y_train_under)
                            elif model_sample == 'exp':
                                if under == 0:
                                    classifier.fit(X_train_experienced,y_train_experienced)
                                else:
                                    classifier.fit(X_train_experienced_under,y_train_experienced_under)
                            elif model_sample == 'cold':
                                if under == 0:
                                    classifier.fit(X_train_cold,y_train_cold)
                                else:
                                    classifier.fit(X_train_cold_under,y_train_cold_under)
                                    
                            perf_df_tmp = performance_df(classifier)
                            perf_df_tmp['Undersampling Flg'] = under
                            perf_df_tmp['Model Sample'] = model_sample
                            
                            perf_df_tmp['Max Depth'] = max_d
                            perf_df_tmp['Learning Rate'] = lr
                            perf_df_tmp['N Estimators'] = n_est
                            perf_df_tmp['Min Samples Leaf'] = min_c
                            perf_df_tmp['Columns Subsample'] = cols
                                    
                            xgb_models_v02 = pd.concat([xgb_models_v02,perf_df_tmp])
                                    
xgb_models_v02.to_csv('xgb_models_v02.csv')

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[15:18:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "min_child_samples" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:18:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[15:18:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "min_child_samples" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:18:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

[19:08:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "min_child_samples" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[19:08:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[19:08:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "min_child_samples" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[19:08:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[19:08:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "min_child_samples" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[19:08:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


* Imputation

In [15]:
imp = SimpleImputer()
imp.fit(X_train)

SimpleImputer()

In [16]:
X_train_imputed = pd.DataFrame(imp.transform(X_train),columns= X_train.columns, index=X_train.index)
X_train_under_imputed = pd.DataFrame(imp.transform(X_train_under),columns= X_train_under.columns, index=X_train_under.index)
X_train_experienced_imputed = pd.DataFrame(imp.transform(X_train_experienced),columns= X_train_experienced.columns, index=X_train_experienced.index)
X_train_experienced_under_imputed = pd.DataFrame(imp.transform(X_train_experienced_under),columns= X_train_experienced_under.columns, index=X_train_experienced_under.index)
X_train_cold_imputed = pd.DataFrame(imp.transform(X_train_cold),columns= X_train_cold.columns, index=X_train_cold.index)
X_train_cold_under_imputed = pd.DataFrame(imp.transform(X_train_cold_under),columns= X_train_cold_under.columns, index=X_train_cold_under.index)

In [17]:
X_val_imputed = pd.DataFrame(imp.transform(X_val),columns= X_val.columns, index=X_val.index)
X_val_experienced_imputed = pd.DataFrame(imp.transform(X_val_experienced),columns= X_val_experienced.columns, index=X_val_experienced.index)
X_val_cold_imputed = pd.DataFrame(imp.transform(X_val_cold),columns= X_val_cold.columns, index=X_val_cold.index)

In [18]:
X_test_imputed = pd.DataFrame(imp.transform(X_test),columns= X_test.columns, index=X_test.index)
X_test_experienced_imputed = pd.DataFrame(imp.transform(X_test_experienced),columns= X_test_experienced.columns, index=X_test_experienced.index)
X_test_cold_imputed = pd.DataFrame(imp.transform(X_test_cold),columns= X_test_cold.columns, index=X_test_cold.index)

In [28]:
def performance_for_rf_df(model):
    
    pred_train = pd.DataFrame(model.predict_proba(X_train_imputed)[:,1],columns=['pred'],index=X_train_imputed.index)
    pred_val = pd.DataFrame(model.predict_proba(X_val_imputed)[:,1],columns=['pred'],index=X_val_imputed.index)
    pred_train = pd.merge(pred_train,y_train,how='left',left_index=True, right_index=True)
    pred_val = pd.merge(pred_val,y_val,how='left',left_index=True, right_index=True)

    pred_train['pred_binary'] = np.where(pred_train['pred']>0.5,1,0)
    pred_val['pred_binary'] = np.where(pred_val['pred']>0.5,1,0)

    roc_train = roc_auc_score(pred_train['female_label'],pred_train['pred'])
    roc_val = roc_auc_score(pred_val['female_label'],pred_val['pred'])

    err_rate1_train = 1-(pred_train[pred_train['female_label']==1]['pred_binary'].sum()/pred_train[pred_train['female_label']==1]['pred_binary'].count())
    err_rate0_train = pred_train[pred_train['female_label']==0]['pred_binary'].sum()/pred_train[pred_train['female_label']==0]['pred_binary'].count()
    err_rate_train = err_rate1_train+err_rate0_train

    err_rate1_val = 1-(pred_val[pred_val['female_label']==1]['pred_binary'].sum()/pred_val[pred_val['female_label']==1]['pred_binary'].count())
    err_rate0_val = pred_val[pred_val['female_label']==0]['pred_binary'].sum()/pred_val[pred_val['female_label']==0]['pred_binary'].count()
    err_rate_val = err_rate1_val+err_rate0_val

    pred_train_experienced = pd.DataFrame(model.predict_proba(X_train_experienced_imputed)[:,1],columns=['pred'],index=X_train_experienced_imputed.index)
    pred_val_experienced = pd.DataFrame(model.predict_proba(X_val_experienced_imputed)[:,1],columns=['pred'],index=X_val_experienced_imputed.index)
    pred_train_experienced = pd.merge(pred_train_experienced,y_train_experienced,how='left',left_index=True, right_index=True)
    pred_val_experienced = pd.merge(pred_val_experienced,y_val_experienced,how='left',left_index=True, right_index=True)

    pred_train_experienced['pred_binary'] = np.where(pred_train_experienced['pred']>0.5,1,0)
    pred_val_experienced['pred_binary'] = np.where(pred_val_experienced['pred']>0.5,1,0)

    roc_train_experienced = roc_auc_score(pred_train_experienced['female_label'],pred_train_experienced['pred'])
    roc_val_experienced = roc_auc_score(pred_val_experienced['female_label'],pred_val_experienced['pred'])

    err_rate1_train_experienced = 1-(pred_train_experienced[pred_train_experienced['female_label']==1]['pred_binary'].sum()/pred_train_experienced[pred_train_experienced['female_label']==1]['pred_binary'].count())
    err_rate0_train_experienced = pred_train_experienced[pred_train_experienced['female_label']==0]['pred_binary'].sum()/pred_train_experienced[pred_train_experienced['female_label']==0]['pred_binary'].count()
    err_rate_train_experienced = err_rate1_train_experienced+err_rate0_train_experienced

    err_rate1_val_experienced = 1-(pred_val_experienced[pred_val_experienced['female_label']==1]['pred_binary'].sum()/pred_val_experienced[pred_val_experienced['female_label']==1]['pred_binary'].count())
    err_rate0_val_experienced = pred_val_experienced[pred_val_experienced['female_label']==0]['pred_binary'].sum()/pred_val_experienced[pred_val_experienced['female_label']==0]['pred_binary'].count()
    err_rate_val_experienced = err_rate1_val_experienced+err_rate0_val_experienced

    pred_train_cold = pd.DataFrame(model.predict_proba(X_train_cold_imputed)[:,1],columns=['pred'],index=X_train_cold_imputed.index)
    pred_val_cold = pd.DataFrame(model.predict_proba(X_val_cold_imputed)[:,1],columns=['pred'],index=X_val_cold_imputed.index)
    pred_train_cold = pd.merge(pred_train_cold,y_train_cold,how='left',left_index=True, right_index=True)
    pred_val_cold = pd.merge(pred_val_cold,y_val_cold,how='left',left_index=True, right_index=True)

    pred_train_cold['pred_binary'] = np.where(pred_train_cold['pred']>0.5,1,0)
    pred_val_cold['pred_binary'] = np.where(pred_val_cold['pred']>0.5,1,0)

    roc_train_cold = roc_auc_score(pred_train_cold['female_label'],pred_train_cold['pred'])
    roc_val_cold = roc_auc_score(pred_val_cold['female_label'],pred_val_cold['pred'])

    err_rate1_train_cold = 1-(pred_train_cold[pred_train_cold['female_label']==1]['pred_binary'].sum()/pred_train_cold[pred_train_cold['female_label']==1]['pred_binary'].count())
    err_rate0_train_cold = pred_train_cold[pred_train_cold['female_label']==0]['pred_binary'].sum()/pred_train_cold[pred_train_cold['female_label']==0]['pred_binary'].count()
    err_rate_train_cold = err_rate1_train_cold+err_rate0_train_cold

    err_rate1_val_cold = 1-(pred_val_cold[pred_val_cold['female_label']==1]['pred_binary'].sum()/pred_val_cold[pred_val_cold['female_label']==1]['pred_binary'].count())
    err_rate0_val_cold = pred_val_cold[pred_val_cold['female_label']==0]['pred_binary'].sum()/pred_val_cold[pred_val_cold['female_label']==0]['pred_binary'].count()
    err_rate_val_cold = err_rate1_val_cold+err_rate0_val_cold

    perf_df = pd.DataFrame({'Train ROC':[roc_train],
                            'Val ROC':[roc_val],
                            'Train BAR':[1-err_rate_train],
                            'Val BAR':[1-err_rate_val],
                            'Train ROC - Exp':[roc_train_experienced],
                            'Val ROC - Exp':[roc_val_experienced],
                            'Train BAR - Exp':[1-err_rate_train_experienced],
                            'Val BAR - Exp':[1-err_rate_val_experienced],
                            'Train ROC - Cold':[roc_train_cold],
                            'Val ROC - Cold':[roc_val_cold],
                            'Train BAR - Cold':[1-err_rate_train_cold],
                            'Val BAR - Cold':[1-err_rate_val_cold]})
    
    return perf_df

### Random Forest

In [30]:
rf_models_v02 = pd.DataFrame()

for under in [0,1]:
    for model_sample in ['all','exp','cold']:
        for max_d in [2,4,10]:
            for lr in ['gini', 'entropy']:
                for n_est in [100,200]:
                    for min_c in [20,50,100]:
                        for cols in [0.5,0.8,1]:
                            
                            classifier = RandomForestClassifier(max_depth=max_d,criterion=lr,n_estimators=n_est,min_samples_leaf=min_c,max_features=cols)
                            if model_sample == 'all':
                                if under == 0:
                                    classifier.fit(X_train_imputed,y_train)
                                else:
                                    classifier.fit(X_train_under_imputed,y_train_under)
                            elif model_sample == 'exp':
                                if under == 0:
                                    classifier.fit(X_train_experienced_imputed,y_train_experienced)
                                else:
                                    classifier.fit(X_train_experienced_under_imputed,y_train_experienced_under)
                            elif model_sample == 'cold':
                                if under == 0:
                                    classifier.fit(X_train_cold_imputed,y_train_cold)
                                else:
                                    classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
                                    
                            perf_df_tmp = performance_for_rf_df(classifier)
                            perf_df_tmp['Undersampling Flg'] = under
                            perf_df_tmp['Model Sample'] = model_sample
                            
                            perf_df_tmp['Max Depth'] = max_d
                            perf_df_tmp['criterion'] = lr
                            perf_df_tmp['N Estimators'] = n_est
                            perf_df_tmp['Min Samples Leaf'] = min_c
                            perf_df_tmp['Columns Subsample'] = cols
                                    
                            rf_models_v02 = pd.concat([rf_models_v02,perf_df_tmp])
                                    
rf_models_v02.to_csv('rf_models_v02.csv')

<ipython-input-30-a539fecadd9c>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-30-a539fecadd9c>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-30-a539fecadd9c>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-30-a539fecadd9c>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-30-a539fecadd9c>:14: DataConversionWarning: A column-

<ipython-input-30-a539fecadd9c>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-30-a539fecadd9c>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-30-a539fecadd9c>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-30-a539fecadd9c>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-30-a539fecadd9c>:14: DataConversionWarning: A column-

<ipython-input-30-a539fecadd9c>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-30-a539fecadd9c>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-30-a539fecadd9c>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-30-a539fecadd9c>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-30-a539fecadd9c>:14: DataConversionWarning: A column-

<ipython-input-30-a539fecadd9c>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_imputed,y_train_experienced)
<ipython-input-30-a539fecadd9c>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_imputed,y_train_experienced)
<ipython-input-30-a539fecadd9c>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_imputed,y_train_experienced)
<ipython-input-30-a539fecadd9c>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_i

<ipython-input-30-a539fecadd9c>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_imputed,y_train_experienced)
<ipython-input-30-a539fecadd9c>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_imputed,y_train_experienced)
<ipython-input-30-a539fecadd9c>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_imputed,y_train_experienced)
<ipython-input-30-a539fecadd9c>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_i

...

<ipython-input-30-a539fecadd9c>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_under_imputed,y_train_experienced_under)
<ipython-input-30-a539fecadd9c>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_under_imputed,y_train_experienced_under)
<ipython-input-30-a539fecadd9c>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_under_imputed,y_train_experienced_under)
<ipython-input-30-a539fecadd9c>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  

<ipython-input-30-a539fecadd9c>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_under_imputed,y_train_experienced_under)
<ipython-input-30-a539fecadd9c>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_under_imputed,y_train_experienced_under)
<ipython-input-30-a539fecadd9c>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_under_imputed,y_train_experienced_under)
<ipython-input-30-a539fecadd9c>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  

<ipython-input-30-a539fecadd9c>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
<ipython-input-30-a539fecadd9c>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
<ipython-input-30-a539fecadd9c>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
<ipython-input-30-a539fecadd9c>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,

<ipython-input-30-a539fecadd9c>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
<ipython-input-30-a539fecadd9c>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
<ipython-input-30-a539fecadd9c>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
<ipython-input-30-a539fecadd9c>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,

<ipython-input-30-a539fecadd9c>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
<ipython-input-30-a539fecadd9c>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
<ipython-input-30-a539fecadd9c>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
<ipython-input-30-a539fecadd9c>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,

In [29]:
rf_models_v03 = pd.DataFrame()

for under in [0,1]:
    for model_sample in ['all','exp','cold']:
        for max_d in [10,20,50,100]:
            for lr in ['gini']:
                for n_est in [100,200,300]:
                    for min_c in [20,50,100]:
                        for cols in [0.5,0.8,1]:
                            
                            classifier = RandomForestClassifier(max_depth=max_d,criterion=lr,n_estimators=n_est,min_samples_leaf=min_c,max_features=cols)
                            if model_sample == 'all':
                                if under == 0:
                                    classifier.fit(X_train_imputed,y_train)
                                else:
                                    classifier.fit(X_train_under_imputed,y_train_under)
                            elif model_sample == 'exp':
                                if under == 0:
                                    classifier.fit(X_train_experienced_imputed,y_train_experienced)
                                else:
                                    classifier.fit(X_train_experienced_under_imputed,y_train_experienced_under)
                            elif model_sample == 'cold':
                                if under == 0:
                                    classifier.fit(X_train_cold_imputed,y_train_cold)
                                else:
                                    classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
                                    
                            perf_df_tmp = performance_for_rf_df(classifier)
                            perf_df_tmp['Undersampling Flg'] = under
                            perf_df_tmp['Model Sample'] = model_sample
                            
                            perf_df_tmp['Max Depth'] = max_d
                            perf_df_tmp['criterion'] = lr
                            perf_df_tmp['N Estimators'] = n_est
                            perf_df_tmp['Min Samples Leaf'] = min_c
                            perf_df_tmp['Columns Subsample'] = cols
                                    
                            rf_models_v03 = pd.concat([rf_models_v03,perf_df_tmp])
                                    
rf_models_v03.to_csv('rf_models_v03.csv')

<ipython-input-29-5bc00a1abab6>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-29-5bc00a1abab6>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-29-5bc00a1abab6>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-29-5bc00a1abab6>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-29-5bc00a1abab6>:14: DataConversionWarning: A column-

<ipython-input-29-5bc00a1abab6>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-29-5bc00a1abab6>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-29-5bc00a1abab6>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-29-5bc00a1abab6>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-29-5bc00a1abab6>:14: DataConversionWarning: A column-

<ipython-input-29-5bc00a1abab6>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-29-5bc00a1abab6>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-29-5bc00a1abab6>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-29-5bc00a1abab6>:14: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_imputed,y_train)
<ipython-input-29-5bc00a1abab6>:14: DataConversionWarning: A column-

<ipython-input-29-5bc00a1abab6>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_imputed,y_train_experienced)
<ipython-input-29-5bc00a1abab6>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_imputed,y_train_experienced)
<ipython-input-29-5bc00a1abab6>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_imputed,y_train_experienced)
<ipython-input-29-5bc00a1abab6>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_i

<ipython-input-29-5bc00a1abab6>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_imputed,y_train_experienced)
<ipython-input-29-5bc00a1abab6>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_imputed,y_train_experienced)
<ipython-input-29-5bc00a1abab6>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_imputed,y_train_experienced)
<ipython-input-29-5bc00a1abab6>:19: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_i

...

<ipython-input-29-5bc00a1abab6>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_under_imputed,y_train_experienced_under)
<ipython-input-29-5bc00a1abab6>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_under_imputed,y_train_experienced_under)
<ipython-input-29-5bc00a1abab6>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_under_imputed,y_train_experienced_under)
<ipython-input-29-5bc00a1abab6>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  

<ipython-input-29-5bc00a1abab6>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_under_imputed,y_train_experienced_under)
<ipython-input-29-5bc00a1abab6>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_under_imputed,y_train_experienced_under)
<ipython-input-29-5bc00a1abab6>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_experienced_under_imputed,y_train_experienced_under)
<ipython-input-29-5bc00a1abab6>:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  

<ipython-input-29-5bc00a1abab6>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
<ipython-input-29-5bc00a1abab6>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
<ipython-input-29-5bc00a1abab6>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
<ipython-input-29-5bc00a1abab6>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,

<ipython-input-29-5bc00a1abab6>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
<ipython-input-29-5bc00a1abab6>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
<ipython-input-29-5bc00a1abab6>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
<ipython-input-29-5bc00a1abab6>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,

<ipython-input-29-5bc00a1abab6>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
<ipython-input-29-5bc00a1abab6>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
<ipython-input-29-5bc00a1abab6>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
<ipython-input-29-5bc00a1abab6>:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  classifier.fit(X_train_cold_under_imputed,

* Scaling

In [14]:
mm = MinMaxScaler()
mm.fit(X_train_imputed)

MinMaxScaler()

In [15]:
X_train_scaled = pd.DataFrame(mm.transform(X_train_imputed),columns= X_train_imputed.columns, index=X_train_imputed.index)
X_train_under_scaled = pd.DataFrame(mm.transform(X_train_under_imputed),columns= X_train_under_imputed.columns, index=X_train_under_imputed.index)
X_train_experienced_scaled = pd.DataFrame(mm.transform(X_train_experienced_imputed),columns= X_train_experienced_imputed.columns, index=X_train_experienced_imputed.index)
X_train_experienced_under_scaled = pd.DataFrame(mm.transform(X_train_experienced_under_imputed),columns= X_train_experienced_under_imputed.columns, index=X_train_experienced_under_imputed.index)
X_train_cold_scaled = pd.DataFrame(mm.transform(X_train_cold_imputed),columns= X_train_cold_imputed.columns, index=X_train_cold_imputed.index)
X_train_cold_under_scaled = pd.DataFrame(mm.transform(X_train_cold_under_imputed),columns= X_train_cold_under_imputed.columns, index=X_train_cold_under_imputed.index)

In [17]:
X_val_scaled = pd.DataFrame(mm.transform(X_val_imputed),columns= X_val_imputed.columns, index=X_val_imputed.index)
X_val_experienced_scaled = pd.DataFrame(mm.transform(X_val_experienced_imputed),columns= X_val_experienced_imputed.columns, index=X_val_experienced_imputed.index)
X_val_cold_scaled = pd.DataFrame(mm.transform(X_val_cold_imputed),columns= X_val_cold_imputed.columns, index=X_val_cold_imputed.index)

In [18]:
X_test_scaled = pd.DataFrame(mm.transform(X_test_imputed),columns= X_test_imputed.columns, index=X_test_imputed.index)
X_test_experienced_scaled = pd.DataFrame(mm.transform(X_test_experienced_imputed),columns= X_test_experienced_imputed.columns, index=X_test_experienced_imputed.index)
X_test_cold_scaled = pd.DataFrame(mm.transform(X_test_cold_imputed),columns= X_test_cold_imputed.columns, index=X_test_cold_imputed.index)

In [19]:
def performance_for_scaled_df(model):
    
    pred_train = pd.DataFrame(model.predict_proba(X_train_scaled)[:,1],columns=['pred'],index=X_train_scaled.index)
    pred_val = pd.DataFrame(model.predict_proba(X_val_scaled)[:,1],columns=['pred'],index=X_val_scaled.index)
    pred_train = pd.merge(pred_train,y_train,how='left',left_index=True, right_index=True)
    pred_val = pd.merge(pred_val,y_val,how='left',left_index=True, right_index=True)

    pred_train['pred_binary'] = np.where(pred_train['pred']>0.5,1,0)
    pred_val['pred_binary'] = np.where(pred_val['pred']>0.5,1,0)

    roc_train = roc_auc_score(pred_train['female_label'],pred_train['pred'])
    roc_val = roc_auc_score(pred_val['female_label'],pred_val['pred'])

    err_rate1_train = 1-(pred_train[pred_train['female_label']==1]['pred_binary'].sum()/pred_train[pred_train['female_label']==1]['pred_binary'].count())
    err_rate0_train = pred_train[pred_train['female_label']==0]['pred_binary'].sum()/pred_train[pred_train['female_label']==0]['pred_binary'].count()
    err_rate_train = err_rate1_train+err_rate0_train

    err_rate1_val = 1-(pred_val[pred_val['female_label']==1]['pred_binary'].sum()/pred_val[pred_val['female_label']==1]['pred_binary'].count())
    err_rate0_val = pred_val[pred_val['female_label']==0]['pred_binary'].sum()/pred_val[pred_val['female_label']==0]['pred_binary'].count()
    err_rate_val = err_rate1_val+err_rate0_val

    pred_train_experienced = pd.DataFrame(model.predict_proba(X_train_experienced_scaled)[:,1],columns=['pred'],index=X_train_experienced_scaled.index)
    pred_val_experienced = pd.DataFrame(model.predict_proba(X_val_experienced_scaled)[:,1],columns=['pred'],index=X_val_experienced_scaled.index)
    pred_train_experienced = pd.merge(pred_train_experienced,y_train_experienced,how='left',left_index=True, right_index=True)
    pred_val_experienced = pd.merge(pred_val_experienced,y_val_experienced,how='left',left_index=True, right_index=True)

    pred_train_experienced['pred_binary'] = np.where(pred_train_experienced['pred']>0.5,1,0)
    pred_val_experienced['pred_binary'] = np.where(pred_val_experienced['pred']>0.5,1,0)

    roc_train_experienced = roc_auc_score(pred_train_experienced['female_label'],pred_train_experienced['pred'])
    roc_val_experienced = roc_auc_score(pred_val_experienced['female_label'],pred_val_experienced['pred'])

    err_rate1_train_experienced = 1-(pred_train_experienced[pred_train_experienced['female_label']==1]['pred_binary'].sum()/pred_train_experienced[pred_train_experienced['female_label']==1]['pred_binary'].count())
    err_rate0_train_experienced = pred_train_experienced[pred_train_experienced['female_label']==0]['pred_binary'].sum()/pred_train_experienced[pred_train_experienced['female_label']==0]['pred_binary'].count()
    err_rate_train_experienced = err_rate1_train_experienced+err_rate0_train_experienced

    err_rate1_val_experienced = 1-(pred_val_experienced[pred_val_experienced['female_label']==1]['pred_binary'].sum()/pred_val_experienced[pred_val_experienced['female_label']==1]['pred_binary'].count())
    err_rate0_val_experienced = pred_val_experienced[pred_val_experienced['female_label']==0]['pred_binary'].sum()/pred_val_experienced[pred_val_experienced['female_label']==0]['pred_binary'].count()
    err_rate_val_experienced = err_rate1_val_experienced+err_rate0_val_experienced

    pred_train_cold = pd.DataFrame(model.predict_proba(X_train_cold_scaled)[:,1],columns=['pred'],index=X_train_cold_scaled.index)
    pred_val_cold = pd.DataFrame(model.predict_proba(X_val_cold_scaled)[:,1],columns=['pred'],index=X_val_cold_scaled.index)
    pred_train_cold = pd.merge(pred_train_cold,y_train_cold,how='left',left_index=True, right_index=True)
    pred_val_cold = pd.merge(pred_val_cold,y_val_cold,how='left',left_index=True, right_index=True)

    pred_train_cold['pred_binary'] = np.where(pred_train_cold['pred']>0.5,1,0)
    pred_val_cold['pred_binary'] = np.where(pred_val_cold['pred']>0.5,1,0)

    roc_train_cold = roc_auc_score(pred_train_cold['female_label'],pred_train_cold['pred'])
    roc_val_cold = roc_auc_score(pred_val_cold['female_label'],pred_val_cold['pred'])

    err_rate1_train_cold = 1-(pred_train_cold[pred_train_cold['female_label']==1]['pred_binary'].sum()/pred_train_cold[pred_train_cold['female_label']==1]['pred_binary'].count())
    err_rate0_train_cold = pred_train_cold[pred_train_cold['female_label']==0]['pred_binary'].sum()/pred_train_cold[pred_train_cold['female_label']==0]['pred_binary'].count()
    err_rate_train_cold = err_rate1_train_cold+err_rate0_train_cold

    err_rate1_val_cold = 1-(pred_val_cold[pred_val_cold['female_label']==1]['pred_binary'].sum()/pred_val_cold[pred_val_cold['female_label']==1]['pred_binary'].count())
    err_rate0_val_cold = pred_val_cold[pred_val_cold['female_label']==0]['pred_binary'].sum()/pred_val_cold[pred_val_cold['female_label']==0]['pred_binary'].count()
    err_rate_val_cold = err_rate1_val_cold+err_rate0_val_cold

    perf_df = pd.DataFrame({'Train ROC':[roc_train],
                            'Val ROC':[roc_val],
                            'Train BAR':[1-err_rate_train],
                            'Val BAR':[1-err_rate_val],
                            'Train ROC - Exp':[roc_train_experienced],
                            'Val ROC - Exp':[roc_val_experienced],
                            'Train BAR - Exp':[1-err_rate_train_experienced],
                            'Val BAR - Exp':[1-err_rate_val_experienced],
                            'Train ROC - Cold':[roc_train_cold],
                            'Val ROC - Cold':[roc_val_cold],
                            'Train BAR - Cold':[1-err_rate_train_cold],
                            'Val BAR - Cold':[1-err_rate_val_cold]})
    
    return perf_df

In [23]:
lr_models_v02 = pd.DataFrame()

for under in [0,1]:
    for model_sample in ['all','exp','cold']:
        for pen in ['l1']:
            for c in [0.001,0.1,0.5,1,2,5,10,20]:                            
                classifier = LogisticRegression(penalty=pen,C=c,solver='saga')
                if model_sample == 'all':
                    if under == 0:
                        classifier.fit(X_train_imputed,y_train)
                    else:
                        classifier.fit(X_train_under_imputed,y_train_under)
                elif model_sample == 'exp':
                    if under == 0:
                        classifier.fit(X_train_experienced_imputed,y_train_experienced)
                    else:
                        classifier.fit(X_train_experienced_under_imputed,y_train_experienced_under)
                elif model_sample == 'cold':
                    if under == 0:
                        classifier.fit(X_train_cold_imputed,y_train_cold)
                    else:
                        classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
                        
                perf_df_tmp = performance_for_scaled_df(classifier)
                perf_df_tmp['Undersampling Flg'] = under
                perf_df_tmp['Model Sample'] = model_sample
                
                perf_df_tmp['penalty'] = pen
                perf_df_tmp['C'] = c
                        
                lr_models_v02 = pd.concat([lr_models_v02,perf_df_tmp])
                                    
lr_models_v02.to_csv('lr_models_v02.csv')

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did n

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


In [26]:
svm_models_v02 = pd.DataFrame()

for under in [0,1]:
    for model_sample in ['all','exp','cold']:
        for ker in ['rbf']:
            for c in [0.001,0.1,0.5,1,2,5,10,20]:                            
                classifier = SVC(kernel=ker,C=c,probability=True)
                if model_sample == 'all':
                    if under == 0:
                        classifier.fit(X_train_imputed,y_train)
                    else:
                        classifier.fit(X_train_under_imputed,y_train_under)
                elif model_sample == 'exp':
                    if under == 0:
                        classifier.fit(X_train_experienced_imputed,y_train_experienced)
                    else:
                        classifier.fit(X_train_experienced_under_imputed,y_train_experienced_under)
                elif model_sample == 'cold':
                    if under == 0:
                        classifier.fit(X_train_cold_imputed,y_train_cold)
                    else:
                        classifier.fit(X_train_cold_under_imputed,y_train_cold_under)
                        
                perf_df_tmp = performance_for_scaled_df(classifier)
                perf_df_tmp['Undersampling Flg'] = under
                perf_df_tmp['Model Sample'] = model_sample
                
                perf_df_tmp['kernel'] = ker
                perf_df_tmp['C'] = c
                        
                svm_models_v02 = pd.concat([svm_models_v02,perf_df_tmp])
                                    
svm_models_v02.to_csv('svm_models_v02.csv')

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-

c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\cidem\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-

### Random Forest with CV

In [19]:
y_train_and_val = pd.concat([y_train,y_val])
X_train_and_val = pd.concat([X_train_imputed,X_val_imputed])

In [20]:
X_train_and_val.shape

(5618, 1539)

In [21]:
y_train_and_val['rand'] = np.random.rand(y_train_and_val.shape[0])
y_train_and_val.sort_values(by='rand',inplace=True)
y_train_and_val['rank'] = y_train_and_val.groupby('female_label').cumcount()+1
y_train_and_val

,female_label,rand,rank
unique_id,,,
7235,1,0.000180,1
7903,0,0.000290,1
3764,1,0.000746,2
6792,1,0.000952,3
7072,1,0.001106,4
...,...,...,...
1239,0,0.999127,1938
4160,0,0.999598,1939
7771,1,0.999624,3677


In [22]:
cnt0 = y_train_and_val[y_train_and_val['female_label']==0].shape[0]
cnt1 = y_train_and_val[y_train_and_val['female_label']==1].shape[0]

In [23]:
def class0_folds(x):
    y = 0
    if x<=cnt0*1/10:
        y = 1
    elif x<=cnt0*2/10:
        y = 2
    elif x<=cnt0*3/10:
        y = 3
    elif x<=cnt0*4/10:
        y = 4
    elif x<=cnt0*5/10:
        y = 5
    elif x<=cnt0*6/10:
        y = 6
    elif x<=cnt0*7/10:
        y = 7
    elif x<=cnt0*8/10:
        y = 8
    elif x<=cnt0*9/10:
        y = 9
    elif x<=cnt0*10/10:
        y = 10
    return y

def class1_folds(x):
    y = 0
    if x<=cnt1*1/10:
        y = 1
    elif x<=cnt1*2/10:
        y = 2
    elif x<=cnt1*3/10:
        y = 3
    elif x<=cnt1*4/10:
        y = 4
    elif x<=cnt1*5/10:
        y = 5
    elif x<=cnt1*6/10:
        y = 6
    elif x<=cnt1*7/10:
        y = 7
    elif x<=cnt1*8/10:
        y = 8
    elif x<=cnt1*9/10:
        y = 9
    elif x<=cnt1*10/10:
        y = 10
    return y

In [24]:
y_train_and_val['class0_fold'] = y_train_and_val['rank'].apply(lambda x: class0_folds(x))
y_train_and_val['class1_fold'] = y_train_and_val['rank'].apply(lambda x: class1_folds(x))
y_train_and_val['fold'] = np.where(y_train_and_val['female_label']==0,y_train_and_val['class0_fold'],y_train_and_val['class1_fold'])
y_train_and_val.drop(['rand','rank','class0_fold','class1_fold'],axis=1,inplace=True)

y_train_and_val

,female_label,fold
unique_id,,
7235,1,1
7903,0,1
3764,1,1
6792,1,1
7072,1,1
...,...,...
1239,0,10
4160,0,10
7771,1,10


In [25]:
y_train_and_val['fold'].value_counts()

2     562
3     562
4     562
5     562
6     562
7     562
8     562
9     562
10    562
1     560
Name: fold, dtype: int64

In [26]:
list(y_train_and_val[y_train_and_val['fold']==1].index)

[7235,
 7903,
 3764,
 6792,
 7072,
 3383,
 654,
 6388,
 5857,
 1325,
 599,
 1741,
 5761,
 5506,
 1713,
 6489,
 2770,
 3814,
 3353,
 4961,
 2802,
 1645,
 3121,
 5701,
 2906,
 4409,
 7635,
 1968,
 6944,
 535,
 2339,
 4508,
 5671,
 166,
 7651,
 7074,
 5785,
 312,
 732,
 5018,
 5602,
 4554,
 1529,
 4506,
 4202,
 3382,
 7937,
 2433,
 1117,
 1915,
 3069,
 4719,
 6582,
 381,
 6979,
 5344,
 5823,
 5809,
 5659,
 1032,
 825,
 5595,
 3233,
 897,
 5850,
 3645,
 7574,
 6923,
 7754,
 4379,
 1478,
 3180,
 280,
 5510,
 7002,
 7482,
 3679,
 2845,
 2494,
 6962,
 3957,
 3545,
 910,
 5417,
 5625,
 7637,
 415,
 5082,
 7425,
 2084,
 5915,
 2849,
 2495,
 1495,
 6590,
 1464,
 5467,
 3900,
 3726,
 2530,
 3942,
 1593,
 7391,
 4599,
 5196,
 7023,
 1775,
 433,
 173,
 4749,
 3732,
 4704,
 1046,
 3322,
 2083,
 616,
 2576,
 6534,
 4037,
 4780,
 3117,
 276,
 5198,
 870,
 2840,
 5485,
 3389,
 236,
 960,
 2823,
 3405,
 6958,
 7749,
 5323,
 2529,
 5739,
 2186,
 4479,
 5472,
 968,
 7469,
 3486,
 3232,
 4081,
 3685,
 1610

In [28]:
fold_df = y_train_and_val[['fold']]
y_train_and_val = y_train_and_val[['female_label']]

In [39]:
var_list = pd.read_csv('variable_list.csv')

In [41]:
var_list.columns=['index','feature']

In [47]:
var_list = [x[2:] for x in list(var_list['feature'].values)]

In [48]:
time_init=time.time()

perf_df_rf = pd.DataFrame()
i = 0
for cw in ['balanced',None]:
    for max_d in [5,10,50]:
        for cri in ['gini']:
            for n_est in [100]:
                for min_c in [20,100]:
                    for cols in [0.8,0.4]:
                        for over in [0,1]:
                        
                            i = i + 1
                            print('Running fit ',i)

                            rf = RandomForestClassifier(max_depth = max_d,class_weight = cw, criterion = cri, n_estimators=n_est, min_samples_leaf = min_c, max_features = cols)

                            perf_df_tmp2 = pd.DataFrame()

                            for fold in range(1,11):
                                X_tr = X_train_and_val[X_train_and_val.index.isin(list(fold_df[fold_df['fold']!=fold].index))][var_list]
                                y_tr = y_train_and_val[y_train_and_val.index.isin(list(fold_df[fold_df['fold']!=fold].index))]

                                X_vl = X_train_and_val[X_train_and_val.index.isin(list(fold_df[fold_df['fold']==fold].index))][var_list]
                                y_vl = y_train_and_val[y_train_and_val.index.isin(list(fold_df[fold_df['fold']==fold].index))]
                                
                                if over == 1:
                                    oversample = SMOTE()
                                    X_tr2, y_tr2 = oversample.fit_resample(X_tr, y_tr)
                                else:
                                    X_tr2 = X_tr.copy()
                                    y_tr2 = y_tr.copy()

                                rf.fit(X_tr2,y_tr2)

                                pred_train = pd.DataFrame(rf.predict_proba(X_tr)[:,1],columns=['pred'],index=X_tr.index)
                                pred_train = pd.merge(pred_train,y_train_and_val[['female_label']],how='left',left_index=True,right_index=True)

                                pred_val = pd.DataFrame(rf.predict_proba(X_vl)[:,1],columns=['pred'],index=X_vl.index)
                                pred_val = pd.merge(pred_val,y_train_and_val[['female_label']],how='left',left_index=True,right_index=True)

                                pred_train['pred_binary'] = np.where(pred_train['pred']>0.5,1,0)
                                pred_val['pred_binary'] = np.where(pred_val['pred']>0.5,1,0)
    
                                perf_df_tmp = pd.DataFrame()

                                roc_train = roc_auc_score(pred_train['female_label'],pred_train['pred'])
                                roc_val = roc_auc_score(pred_val['female_label'],pred_val['pred'])

                                err_rate1_train = 1-(pred_train[pred_train['female_label']==1]['pred_binary'].sum()/pred_train[pred_train['female_label']==1]['pred_binary'].count())
                                err_rate0_train = pred_train[pred_train['female_label']==0]['pred_binary'].sum()/pred_train[pred_train['female_label']==0]['pred_binary'].count()
                                err_rate_train = err_rate1_train+err_rate0_train

                                err_rate1_val = 1-(pred_val[pred_val['female_label']==1]['pred_binary'].sum()/pred_val[pred_val['female_label']==1]['pred_binary'].count())
                                err_rate0_val = pred_val[pred_val['female_label']==0]['pred_binary'].sum()/pred_val[pred_val['female_label']==0]['pred_binary'].count()
                                err_rate_val = err_rate1_val+err_rate0_val
                                
                                perf_df_tmp = pd.DataFrame({'Train ROC':[roc_train]})
                                perf_df_tmp['Val ROC'] = roc_val
                                perf_df_tmp['Error Train'] = err_rate_train
                                perf_df_tmp['Error Val'] = err_rate_val
                                perf_df_tmp['fold'] = fold
                                perf_df_tmp['dummy'] = 1

                                perf_df_tmp2 = pd.concat([perf_df_tmp2,perf_df_tmp])

                            perf_df_tmp2 = perf_df_tmp2.groupby('dummy').agg({'Train ROC':[np.mean],'Val ROC':[np.mean],'Error Train':[np.mean],'Error Val':[np.mean]})
                            perf_df_tmp2.columns=['Train ROC','Val ROC','Error Train','Error Val']
                            perf_df_tmp2.reset_index(inplace=True)


                            perf_df_tmp2['max_depth'] = max_d
                            perf_df_tmp2['class_weight'] = cw 
                            perf_df_tmp2['criterion'] = cri
                            perf_df_tmp2['n_estimators'] = n_est
                            perf_df_tmp2['min_samples_leaf'] = min_c
                            perf_df_tmp2['max_features'] = cols
                            perf_df_tmp2['oversampling_flag'] = over

                            perf_df_rf = pd.concat([perf_df_rf,perf_df_tmp2])
                        
perf_df_rf.to_csv('perf_df_rf_cv.csv')
perf_df_rf.head()

time_final=time.time()
print('time passed in minutes:',(time_final-time_init)/60)

Running fit  1


<ipython-input-48-4adc2a1dbfbb>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_tr2,y_tr2)
<ipython-input-48-4adc2a1dbfbb>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_tr2,y_tr2)
<ipython-input-48-4adc2a1dbfbb>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_tr2,y_tr2)
<ipython-input-48-4adc2a1dbfbb>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_tr2,y_tr2)
<ipython-input-48-4adc2a1dbfbb>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y t

Running fit  2


<ipython-input-48-4adc2a1dbfbb>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_tr2,y_tr2)
<ipython-input-48-4adc2a1dbfbb>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_tr2,y_tr2)
<ipython-input-48-4adc2a1dbfbb>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_tr2,y_tr2)
<ipython-input-48-4adc2a1dbfbb>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_tr2,y_tr2)
<ipython-input-48-4adc2a1dbfbb>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y t

Running fit  3


...

Running fit  42


<ipython-input-48-4adc2a1dbfbb>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_tr2,y_tr2)
<ipython-input-48-4adc2a1dbfbb>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_tr2,y_tr2)
<ipython-input-48-4adc2a1dbfbb>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_tr2,y_tr2)
<ipython-input-48-4adc2a1dbfbb>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_tr2,y_tr2)
<ipython-input-48-4adc2a1dbfbb>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y t

Running fit  43


<ipython-input-48-4adc2a1dbfbb>:34: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_tr2,y_tr2)


KeyboardInterrupt: 

In [49]:
perf_df_rf

,dummy,Train ROC,Val ROC,Error Train,Error Val,max_depth,class_weight,criterion,n_estimators,min_samples_leaf,max_features,oversampling_flag
0,1,0.508114,0.499473,1.002957,0.999427,5,balanced,gini,100,20,0.8,0
0,1,0.488078,0.473462,1.031211,1.023675,5,balanced,gini,100,20,0.8,1
0,1,0.510915,0.496209,0.999120,1.016040,5,balanced,gini,100,20,0.4,0
0,1,0.480984,0.460916,1.039991,1.048528,5,balanced,gini,100,20,0.4,1
0,1,0.509996,0.508681,0.991733,0.984858,5,balanced,gini,100,100,0.8,0
0,1,0.469968,0.463959,1.046269,1.033988,5,balanced,gini,100,100,0.8,1
0,1,0.506473,0.500045,0.997747,1.009307,5,balanced,gini,100,100,0.4,0
0,1,0.468272,0.458569,1.047245,1.048983,5,balanced,gini,100,100,0.4,1
0,1,0.526603,0.518251,0.989180,0.991754,10,balanced,gini,100,20,0.8,0
0,1,0.508417,0.483963,1.019489,1.008686,10,balanced,gini,100,20,0.8,1
